# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [5]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key se ve muy bien")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
openai = OpenAI()

API key se ve muy bien


In [47]:
#Prompt del sistema
system_prompt = "Eres un experto en LLM, inteligencia artificial y python \
el cual se le van a proporcionar una o varias preguntas técnicas relacionadas a los temas a los que eres experto \
y tienes que responder de tal manera que un estudiante que es muy novato y apenas está aprendiendo entienda"

In [49]:
# here is the question; type over this to ask something new

question = """
Por favor explica que hace este código y porque:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [51]:
# Prompt del usuario
user_prompt = f"Responde la siguiente pregunta en formato markdown: {question}"

In [94]:
# Crear la repuesta sin streaming
def crear_respuesta():
    response = openai.chat.completions.create(
        model = MODEL_GPT,
        messages = [
            {"role":"system", "content": system_prompt},
            {"role":"user", "content": user_prompt}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [104]:
crear_respuesta()

Claro, te explicaré qué hace este código en Python de manera sencilla.

### Explicación del código

El código que compartiste es:

```python
yield from {book.get("author") for book in books if book.get("author")}
```

#### Desglose del código:

1. **`{... for book in books if book.get("author")}`**:
   - Esta parte es una **comprensión de conjunto** (set comprehension). Se utiliza para crear un conjunto (set) nuevo a partir de una colección existente, que en este caso es `books`.
   - Aquí, `books` se supone que es una lista (o cualquier iterable) que contiene libros, y cada libro es representado como un diccionario (un conjunto de pares clave-valor).
   - **`book.get("author")`** intenta obtener el autor del libro. La función `get` es segura porque si la clave "author" no existe en el diccionario, devolverá `None` en lugar de causar un error.
   - **`if book.get("author")`** asegura que solo se incluyan en el conjunto aquellos libros que efectivamente tienen un autor. Es decir, filtra los libros y sólo agrega a los que tienen un valor no nulo para "author".

2. **`yield from`**:
   - `yield from` es una palabra clave en Python que se utiliza dentro de una función generadora. Lo que hace es delegar la generación de valores a otro generador o iterable.
   - En este caso, está "generando" los autores que fueron extraídos del conjunto que acabamos de crear.

#### ¿Qué sucede en resumen?

1. Se pasa por cada libro en la lista `books`.
2. Se extraen los autores de los libros que tienen un autor definido.
3. Se crea un conjunto de esos autores (un conjunto no tiene duplicados).
4. Finalmente, `yield from` permite que la función devuelva cada autor uno por uno cuando sea llamada.

### ¿Por qué se utiliza este código?

- **Eficiencia**: Al usar un conjunto, se asegura que cada autor aparezca solo una vez, eliminando duplicados.
- **Generadores**: Al usar `yield from`, se puede manejar la salida de una manera más eficiente, permitiendo que el código sea más fácil de utilizar en bucles o cualquier otro lugar en el que se necesiten los autores.

### Conclusión

Este código es útil para extraer y liberar (uno por uno) una lista única de autores de una lista de libros, asegurando que no haya repeticiones. Esto es muy común en el procesamiento de datos, donde queremos filtrar y generar información de manera eficiente.

In [167]:
# Get gpt-4o-mini to answer, with streaming
def stream_respuesta():
    stream = openai.chat.completions.create(
        model = MODEL_GPT,
        messages = [
            {"role":"system", "content": system_prompt},
            {"role":"user", "content": user_prompt}
        ],
        stream = True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        #response = response.replace("```", "").replace("markdown","")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [169]:
stream_respuesta()

Claro, voy a desglosar el código que has proporcionado para que sea fácil de entender.

```python
yield from {book.get("author") for book in books if book.get("author")}
```

### Explicación del Código

1. **Contexto General**:
   - Este código es parte de una función generadora en Python, que usa `yield` para devolver un valor. Las funciones generadoras permiten producir una serie de valores a lo largo del tiempo, sin tener que generar todos a la vez.

2. **Comprensión de Conjuntos**:
   - La parte `{book.get("author") for book in books if book.get("author")}` es una **comprensión de conjuntos** (set comprehension). Esto crea un nuevo conjunto (un tipo de colección en Python) que contiene ciertos elementos.

   - **¿Qué hace este fragmento específicamente?**:
     - `for book in books`: Aquí se está iterando sobre una lista llamada `books`, donde cada `book` es un elemento de esa lista (normalmente, se espera que sea un diccionario que representa un libro).
     - `book.get("author")`: Esto intenta obtener el valor asociado a la clave `"author"` del diccionario `book`. Si no existe, devuelve `None`.
     - `if book.get("author")`: Esta parte filtra los libros, de modo que solo se incluyan aquellos que tienen un autor definido (es decir, no deben ser `None`).

   - Al final, todo esto genera un conjunto de autores de los libros.

3. **Uso de `yield from`**:
   - `yield from` permite a una función generadora devolver cada elemento de un iterable (en este caso, el conjunto de autores) uno por uno.
   - En este código, se está diciendo "devuelve cada autor del conjunto que hemos creado".

### Resumen

- Este código genera un conjunto de autores a partir de una lista de libros, excluyendo los que no tienen un autor definido, y luego devuelve cada autor uno por uno en una función generadora.

### Ejemplo Visual

Imagina que tienes la siguiente lista de libros:

```python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": None},
    {"title": "Book 3", "author": "Author B"},
]
```

Con el código, obtendrías:

- Un conjunto de autores: `{"Author A", "Author B"}`.
- Y cuando llames a la función generadora, te devolverá `"Author A"` y luego `"Author B"`.

Espero que esta explicación te haya ayudado a entender el código. Si tienes más preguntas, no dudes en preguntar.

In [186]:
# Get Llama 3.2 to answer
import ollama

def stream_respuesta_ollama():
    stream = ollama.chat(
        model = MODEL_LLAMA,
        messages = [
            {"role":"system", "content": system_prompt},
            {"role":"user", "content": user_prompt}
        ],
        stream = True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk['message']['content'] or ''
        #response = response.replace("```", "").replace("markdown","")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [188]:
stream_respuesta_ollama()

**Análisis del Código**

El código proporcionado es una expresión generadora en Python, utilizada para obtener los autores de una lista de libros (`books`). A continuación, se explica qué hace cada parte del código y por qué.

### **`yield from`**

La palabra clave `yield from` es un método de generación similar a `yield`, pero con algunas diferencias importantes. En lugar de producir un valor directamente en la expresión, `yield from` devuelve una especie de "llamada" a otra expresión, que ejecuta y devolverá sus valores.

En este caso, `yield from` se utiliza para generar los autores de cada libro en la lista (`books`). En lugar de tener que utilizar un bucle (`for`) o expresiones condicionales para obtener el autor correcto, podemos aprovechar el método `get()` de la lista para buscar directamente el valor deseado.

### **`{book.get("author") for book in books if book.get("author")}`**

La expresión entre llaves es una expresión generadora que se utiliza para generar los autores de cada libro en la lista. A continuación, se explica cómo funciona:

*   **`for book in books`**: Este bucle recorre cada elemento (`book`) en la lista (`books`).
*   **`if book.get("author")`**: En cada iteración del bucle, se verifica si el libro tiene un autor utilizando el método `get()` con la clave `"author"`. Si el autor no existe, el valor será `None`.
*   **`book.get("author")`**: Si el autor existe, se devuelve su valor (el nombre real del autor).

**Por qué `yield from` es utilizado aquí?**

En este caso, podemos utilizar `yield from` porque la expresión dentro de las llaves (`{book.get("author") for book in books if book.get("author")}`) ya utiliza un bucle para recorrer los libros y obtener sus autores. Por lo tanto, en lugar de tener que utilizar otro bucle o expresiones condicionales, podemos aprovechar el método `yield from` para generar los autores de manera más concisa.

**Resumen**

El código utiliza la expresión generadora `{book.get("author") for book in books if book.get("author")}` para obtener los autores de cada libro en la lista. La palabra clave `yield from` se utiliza para generar esta expresión y producir sus valores directamente en lugar de tener que utilizar un bucle o expresiones condicionales adicionales.

**Ejemplo con una lista de libros**

```python
books = [
    {"title": "Libro 1", "author": "Author A"},
    {"title": "Libro 2", "author": "Author B"},
    {"title": "Libro 3"}
]

autores = yield from {book.get("author") for book in books if book.get("author")}
print(autores)  # Output: ['Author A', 'Author B']
```

En este ejemplo, se define una lista de libros (`books`) con algunos autores y título. Luego, la expresión generadora es utilizada para obtener los autores de cada libro y imprimirlos en el console.